In [2]:
import numpy as np
import numpy.matlib as ml
import numba
import matplotlib.pyplot as plt

In [30]:
@numba.njit
def calculate_likelihood_c1(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP):
    # likelihood P(Xv, Xa|C =1)
    
    firstDenom = 2*np.pi*np.sqrt(varV*varA + varV*varP +varA*varP)
    firstTerm = 1/firstDenom 
    secondNum = (Xv - Xa)**2 * varP + (Xv -0)**2 * varA + (Xa - 0)**2* varV 
    secondDenom = (varV * varA) + (varV * varP) + (varA * varP)
    secondTerm = np.exp((-0.5*(secondNum/secondDenom)))
    likelihood_com = firstTerm*secondTerm

    return likelihood_com

@numba.njit
def calculate_likelihood_c2(Xv,Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP):
    # likelihood P(Xv, Xa|C =2)
    
    firstTerm = 2*np.pi*np.sqrt((varV + varP)*(varA+varP))
    secondTerm1 = (Xv - 0)**2/(varV + varP)
    secondTerm2 = (Xa - 0)**2 / (varA + varP)
    secondTermFull = np.exp((-0.5*(secondTerm1+secondTerm2)) )
    likelihood_ind = secondTermFull/firstTerm

    return likelihood_ind

@numba.njit
def calculate_posterior(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP):
    # p(C = 1|Xv,Xa) posterior
    
    likelihood_common = calculate_likelihood_c1(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)
    likelihood_ind = calculate_likelihood_c2(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)
    post_common = likelihood_common * pCommon 
    post_indep = likelihood_ind * (1-pCommon)
    posterior = post_common/(post_common +post_indep)

    return posterior

def opt_position_conditionalised_C1(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP):
    # Get optimal location given C = 1
    
    cues = Xv/varV + Xa/varA + ml.repmat(pCommon,N,1)/varP
    inverseVar = 1/varV + 1/varA + 1/varP
    sHatC1 = cues/inverseVar

    return sHatC1

def opt_position_conditionalised_C2(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP):
        # Get optimal locationS given C = 2
        
    visualCue = Xv/varV +ml.repmat(pCommon,N,1)/varP
    visualInvVar = 1/varV + 1/ varP
    sHatVC2 = visualCue/visualInvVar
    audCue = Xa/varA + ml.repmat(pCommon,N,1)/varP
    audInvVar = 1/varA + 1/ varP
    sHatAC2 = audCue/audInvVar

    return sHatVC2, sHatAC2

def optimal_visual_location(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP):
    # Use Model Averaging to compute final visual est
    
    posterior_1C = calculate_posterior(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)
    sHatVC1 = opt_position_conditionalised_C1(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)
    sHatVC2 = opt_position_conditionalised_C2(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)[0]
    sHatV = posterior_1C*sHatVC1 + (1-posterior_1C)*sHatVC2 #model averaging

    return sHatV

def optimal_aud_location(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP):
    # Use Model Averaging to compute final auditory est
    
    posterior_1C = calculate_posterior(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)
    sHatAC1 = opt_position_conditionalised_C1(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)
    sHatAC2 = opt_position_conditionalised_C2(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)[1]
    sHatA = posterior_1C*sHatAC1 + (1-posterior_1C)*sHatAC2 #model averaging
    return sHatA


In [136]:
N = 10000
pCommon, sigV, sigA, sigP = 0.27, 1, 1, 10
varV, varA, varP = sigV**2, sigA**2, sigP**2
vloc, aloc = 24, 24

Xv, Xa = sigV * np.random.randn(N,1) + vloc, sigA * np.random.randn(N,1) + aloc

In [137]:

sHatA = optimal_aud_location(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)

In [76]:
@numba.njit(fastmath = True)
def calculate_posterior(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP):
    # p(C = 1|Xv,Xa) posterior
    
    likelihood_common = calculate_likelihood_c1(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)
    likelihood_ind = calculate_likelihood_c2(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)
    post_common = likelihood_common * pCommon 
    post_indep = likelihood_ind * (1-pCommon)
    posterior = post_common/(post_common +post_indep)
    #plt.hist(posterior)
    #plt.title("posterior")
    #plt.show()
    return posterior

In [78]:
%%timeit
calculate_posterior(Xv, Xa, N, pCommon, sigV, varV, sigA, varA, sigP, varP)

195 µs ± 7.94 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [80]:
def count():

    pass

@numba.njit
def bin(y, possible_locations):
    return min(possible_locations, key = lambda x:abs(x-y))

@numba.njit
def binner(sHatA, possible_locations): 
    l = []
    for i in sHatA:
        y = min(possible_locations, ley=lambda x:np.abs(x-1))
        l.append(y)
    return l

In [106]:
@numba.njit
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


def binner(sHatA, possible_locations):
    poss = np.asarray(possible_locations)
    print(poss)
    binned = (np.abs(sHatA - possible_locations))
    print(type(binned))
    return binned

[-24 -12   0  12  24]
<class 'numpy.ndarray'>


array([[24.08354122, 12.08354122,  0.08354122, 11.91645878, 23.91645878],
       [24.50464859, 12.50464859,  0.50464859, 11.49535141, 23.49535141],
       [23.48658195, 11.48658195,  0.51341805, 12.51341805, 24.51341805],
       ...,
       [23.95990026, 11.95990026,  0.04009974, 12.04009974, 24.04009974],
       [23.92531249, 11.92531249,  0.07468751, 12.07468751, 24.07468751],
       [23.94966072, 11.94966072,  0.05033928, 12.05033928, 24.05033928]])

In [109]:
def g(possible_locations, sHatV):
    return possible_locations[np.abs(np.subtract.outer(sHatV, bins)).argmin(axis=1)]

g(np.array([-24, -12, 0, 12, 24]), sHatA)

NameError: name 'bins' is not defined

In [141]:
#@numba.njit
def custom_round(arr, bins):
    bin_centers = (bins[:-1] + bins[1:])/2 
    idx = np.digitize(arr, bin_centers)
    result = bins[idx]
    return result

%%timeit custom_round(sHatA, np.array([-24, -12, 0, 12, 24]))

UsageError: Line magic function `%%timeit` not found.


In [13]:
arr7, arr2 = arr3

In [7]:
arr1 = np.ndarray([20, 20])
arr2 = np.ndarray([1, 10])
arr3 = np.array([arr1, arr2], dtype = object)

In [15]:
arr7.shape

(20, 20)

In [17]:
import numba
@numba.njit
def get_samples(N, vloc, aloc, pCommon, sigV, varV, sigA, varA, sigP, varP):
    return sigV * np.random.randn(N,1) + vloc, sigA * np.random.randn(N,1) + aloc

In [19]:
from bcimodel import bcifit

In [ ]:
bcifit()